In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.7 MB/s eta 0:00:00


In [2]:
from PIL import Image
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib

In [3]:
def derive_key(password):
    return hashlib.sha256(password.encode()).digest()[:16]

In [4]:
def encrypt_message(message, password):
    key = derive_key(password)
    cipher = AES.new(key, AES.MODE_CBC)
    ct = cipher.encrypt(pad(message.encode(), AES.block_size))
    return cipher.iv + ct

In [5]:
def decrypt_message(encrypted_data, password):
    key = derive_key(password)
    iv = encrypted_data[:16]
    ct = encrypted_data[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ct), AES.block_size).decode()


In [9]:
def encode_lsb(image_path, output_path, message, password):
    image = Image.open(image_path)
    img_array = np.array(image)
    x_enc = img_array.copy()

    encrypted_bytes = encrypt_message(message, password)
    print(f"Encrypted message (hex): {encrypted_bytes.hex()}")
    binary_data = ''.join(format(byte, '08b') for byte in encrypted_bytes)
    data_len = len(binary_data)

    n, m, z = 0, 0, 0
    max_n, max_m, max_z = img_array.shape
    if data_len > (max_n * max_m * max_z):
        raise ValueError("Message too large for this image")

    for i, bit in enumerate(binary_data):
        org_val = x_enc[n, m, z]
        new_val = (org_val & 0xFE) | int(bit)
        x_enc[n, m, z] = new_val
        print(f"Embedding bit {bit} at ({n},{m},{z}) original={org_val} new={new_val}")

        z += 1
        if z == max_z:
            z = 0
            m += 1
            if m == max_m:
                m = 0
                n += 1

    encoded_img = Image.fromarray(x_enc)
    encoded_img.save(output_path)
    print(f"Message hidden in {output_path}")

In [10]:
def decode_lsb(image_path, password, num_bytes):
    image = Image.open(image_path)
    img_array = np.array(image)

    n, m, z = 0, 0, 0
    max_n, max_m, max_z = img_array.shape

    bits = ''
    for _ in range(num_bytes * 8):
        bit = str(img_array[n, m, z] & 1)
        bits += bit
        print(f"Extracted bit {bit} from ({n},{m},{z})")

        z += 1
        if z == max_z:
            z = 0
            m += 1
            if m == max_m:
                m = 0
                n += 1

    encrypted_bytes = bytes(int(bits[i:i+8], 2) for i in range(0, len(bits), 8))
    print(f"Encrypted data extracted (hex): {encrypted_bytes.hex()}")

    message = decrypt_message(encrypted_bytes, password)
    print(f"Hidden message: {message}")
    return message

In [11]:
from PIL import Image
img = Image.open("mountains-near-water.jpg")
img.save("mountains-near-water.png")


In [12]:
msg = input("Enter message to hide: ")
pwd = input("Enter password: ")
encode_lsb("mountains-near-water.png", "stego.png", msg, pwd)

pwd2 = input("Enter password for decryption: ")
enc_len = len(encrypt_message(msg, pwd))
decode_lsb("stego.png", pwd2,enc_len)


Enter message to hide: hello
Enter password: 123
Encrypted message (hex): 31884970f9982890df50af0d24b2f79e1193d72773a8cdfa5dea89082757d6d2
Embedding bit 0 at (0,0,0) original=22 new=22
Embedding bit 0 at (0,0,1) original=31 new=30
Embedding bit 1 at (0,0,2) original=4 new=5
Embedding bit 1 at (0,1,0) original=5 new=5
Embedding bit 0 at (0,1,1) original=14 new=14
Embedding bit 0 at (0,1,2) original=0 new=0
Embedding bit 0 at (0,2,0) original=1 new=0
Embedding bit 1 at (0,2,1) original=10 new=11
Embedding bit 1 at (0,2,2) original=0 new=1
Embedding bit 0 at (0,3,0) original=22 new=22
Embedding bit 0 at (0,3,1) original=31 new=30
Embedding bit 0 at (0,3,2) original=4 new=4
Embedding bit 1 at (0,4,0) original=45 new=45
Embedding bit 0 at (0,4,1) original=53 new=52
Embedding bit 0 at (0,4,2) original=29 new=28
Embedding bit 0 at (0,5,0) original=49 new=48
Embedding bit 0 at (0,5,1) original=57 new=56
Embedding bit 1 at (0,5,2) original=33 new=33
Embedding bit 0 at (0,6,0) original=39 new=38

'hello'